In [2]:
# !pip install spacy # install spaCy
# !pip install tqdm # install tqdm package to display the progress
# !pip install textblob # for dictionary-based sentment analysis
# !pip install nrclex # for dictionary-based emotions analysis
# !pip install gensim # for word2vec and doc2vec 
!pip uninstall tensorflow -y
!pip install tensorflow==2.15.0 # install tensorflow for deep learning
!pip install keras # install keras for deep learning

Found existing installation: tensorflow 2.15.0
Uninstalling tensorflow-2.15.0:
  Successfully uninstalled tensorflow-2.15.0
  Obtaining dependency information for tensorflow==2.15.0 from https://files.pythonhosted.org/packages/93/c0/a774286d0383419f558deb27096e5de9f9facd6c27df8e9f9af6fba2f77e/tensorflow-2.15.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached tensorflow-2.15.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.4 kB)
Using cached tensorflow-2.15.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (475.3 MB)


In [3]:
# Import other packages:

import os 
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm


import sklearn
from sklearn import tree
from sklearn.metrics import * 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier

from gensim.models import Word2Vec

# tensorflow imports:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import losses
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical, plot_model

from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, Normalization, BatchNormalization
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras import layers
from keras.backend import clear_session
from keras.utils import pad_sequences
from keras.optimizers import *
from keras.preprocessing import sequence

%matplotlib inline
plt.style.use('fivethirtyeight')

In [4]:
!pip install bertopic

In [30]:
df = pd.read_excel("pm_txn_filt.xlsx")

In [31]:
#Only material trxns
df = df[df["subledgertype"] == "Material"]

In [7]:
df.shape

(183331, 17)

In [8]:
df.head(50)

,sourcetransactionid,workorderid,wophaseid,transactiondate,transactiontype,transactionamount,transactiondescription,subledgertype,phdesc,phshop,phshopwithdesc,division,organization,orgdescription,shopzone,shopzonedescription,shopzonetype
2,614480,2120558,1,2014-01-06 12:23:36.660,INVENTORY MATERIAL RELEASE TO WORK ORDER,48.29,Lithonia Emergency Unit-ELM2 DL,Material,HOUSING WEEKLY INSPECTIONS - EMMET HOUSE (EMME...,110,110 (FM-HOUSING MAINTENANCE 110),CH20131 FM-Operations,CC0537,CC0537 FM-Housing Maintenance,MZHOUSING2,FM-HOUSING MAINTENANCE AREA 2,MAINTENANCE
3,614481,2120559,1,2014-01-06 12:27:20.077,INVENTORY MATERIAL RELEASE TO WORK ORDER,48.29,Lithonia Emergency Unit-ELM2 DL,Material,HOUSING WEEKLY INSPECTIONS - PAGE HOUSE (PAGE ...,110,110 (FM-HOUSING MAINTENANCE 110),CH20131 FM-Operations,CC0537,CC0537 FM-Housing Maintenance,MZHOUSING2,FM-HOUSING MAINTENANCE AREA 2,MAINTENANCE
4,614482,2120559,1,2014-01-06 12:27:20.200,INVENTORY MATERIAL RELEASE TO WORK ORDER,34.28,Fixture Three-Light 16-3/4 Sandstone MW #325375,Material,HOUSING WEEKLY INSPECTIONS - PAGE HOUSE (PAGE ...,110,110 (FM-HOUSING MAINTENANCE 110),CH20131 FM-Operations,CC0537,CC0537 FM-Housing Maintenance,MZHOUSING2,FM-HOUSING MAINTENANCE AREA 2,MAINTENANCE
10,616159,2119390,1,2014-01-07 11:10:26.040,EXTERNAL CHARGE,38.63,"SUPPLIES, BUILDING R&M - FERGUSON ENTERPRISES ...",Material,HOUSING WEEKLY INSPECTIONS - LAMBETH FIELD #52...,141,141 (FM-ARTS PRECINCT ZONE (INACTIVE)),NaN,NaN,NaN,NaN,NaN,NaN
11,616160,2119390,1,2014-01-07 11:10:49.207,EXTERNAL CHARGE,52.84,"SUPPLIES, BUILDING R&M - FERGUSON ENTERPRISES ...",Material,HOUSING WEEKLY INSPECTIONS - LAMBETH FIELD #52...,141,141 (FM-ARTS PRECINCT ZONE (INACTIVE)),NaN,NaN,NaN,NaN,NaN,NaN
12,616543,2120550,1,2014-01-07 15:33:05.363,INVENTORY MATERIAL RELEASE TO WORK ORDER,48.29,Lithonia Emergency Unit-ELM2 DL,Material,HOUSING WEEKLY INSPECTIONS - KENT HOUSE (KENT ...,110,110 (FM-HOUSING MAINTENANCE 110),CH20131 FM-Operations,CC0537,CC0537 FM-Housing Maintenance,MZHOUSING2,FM-HOUSING MAINTENANCE AREA 2,MAINTENANCE
13,616604,2120551,1,2014-01-07 16:10:36.923,INVENTORY MATERIAL RELEASE TO WORK ORDER,48.29,Lithonia Emergency Unit-ELM2 DL,Material,HOUSING WEEKLY INSPECTIONS - DABNEY HOUSE (DAB...,110,110 (FM-HOUSING MAINTENANCE 110),CH20131 FM-Operations,CC0537,CC0537 FM-Housing Maintenance,MZHOUSING2,FM-HOUSING MAINTENANCE AREA 2,MAINTENANCE
24,620395,1506497,1,2014-01-09 14:30:01.197,INVENTORY MATERIAL RELEASE TO WORK ORDER,3.30,Bleach Clorox,Material,"PUMP,VACUUM NO.1 MCLEOD HALL BLDG.0105",371,371 (FM-HSPP ZONE 2 MECHANICAL 371),CH20132 FM-UVA Health Operations,CC0532,CC0532 FM-UVA Health Zone 2,MZHSPP2,HSPP MAINTENANCE ZONE 2,MAINTENANCE
25,620396,1506497,1,2014-01-09 14:30:01.337,INVENTORY MATERIAL RELEASE TO WORK ORDER,14.85,Wiper Lint Free 50,Material,"PUMP,VACUUM NO.1 MCLEOD HALL BLDG.0105",371,371 (FM-HSPP ZONE 2 MECHANICAL 371),CH20132 FM-UVA Health Operations,CC0532,CC0532 FM-UVA Health Zone 2,MZHSPP2,HSPP MAINTENANCE ZONE 2,MAINTENANCE
26,620732,2120551,1,2014-01-09 15:49:40.880,INVENTORY MATERIAL RELEASE TO WORK ORDER,4.99,"Rubber Test End Cap 4""",Material,HOUSING WEEKLY INSPECTIONS - DABNEY HOUSE (DAB...,110,110 (FM-HOUSING MAINTENANCE 110),CH20131 FM-Operations,CC0537,CC0537 FM-Housing Maintenance,MZHOUSING2,FM-HOUSING MAINTENANCE AREA 2,MAINTENANCE


In [9]:
df.columns

Index(['sourcetransactionid', 'workorderid', 'wophaseid', 'transactiondate',
       'transactiontype', 'transactionamount', 'transactiondescription',
       'subledgertype', 'phdesc', 'phshop', 'phshopwithdesc', 'division',
       'organization', 'orgdescription', 'shopzone', 'shopzonedescription',
       'shopzonetype'],
      dtype='object')

In [32]:
##Remove numbers
import re

def remove_numbers(text):
    return re.sub(r'\d+', '', text)

In [11]:
column_string = df['transactiondescription'].astype(str).str.upper()

clean_df_final= column_string.apply(remove_numbers)
clean_df_final_id = pd.DataFrame({
    'sourcetransactionid': df['sourcetransactionid'],
    'transactiondescription': clean_df_final
})

In [12]:
clean_df_final_small= clean_df_final_id['transactiondescription'].drop_duplicates().str.upper()

In [12]:
# import and load spaCy:

import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
from spacy import displacy
spacy.cli.download("en_core_web_sm")
nlp = spacy.load('en_core_web_sm') 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 53.9 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [20]:
# Add new stop words: suppliers and filler words
customize_stop_words = [
    'INVOICE','INVOICES','INV','SUPPORT','SERVICE','SERVICES','SUPPLY', 'BUILDING','PO','JOHNSON','GRAINGER','NOLAND'
    ,'PALMETTO','SCOTSMAN','GMSA','CREX','TRISTATE','CAPITAL','CHARLOTTESVILLE','GARLAND','BLGD','TKT'
    ,'FPSMC','FIDELITY','ENGINEERING CORP','SMITH JMSQC JAMEL','ROBERT','HEROLD','COMPANY','JAMEL','JMSQC'
    , 'CHARLOTTESVI', 'VIRGINIA', 'BRIAN','BRANDON','BLBR','AAF','CARDINAL HOME CENTER','BOA','APEX CONTROLS','DOMINION'
    ,'QUINCY','APEX','DBA','LLC','HOLIDAY INN','NORTON','BUTLER','CARDINAL','HOME CENTER','CORP'
]

# Mark them as stop words
for w in customize_stop_words:
    nlp.vocab[w].is_stop = True

In [22]:
# Add new stop words: intervals (monthly, weekly, annual, etc) and buildings' proper names (e.g, Rouss-Robertson)  

stop_words = pd.read_csv("Building name and time stop words.csv")

# Mark them as stop words
for w in stop_words['words']:
    nlp.vocab[w].is_stop = True

In [15]:
tqdm.pandas() # To display the progress
clean_df_final_small['pr_transactiondescription'] = clean_df_final_small.progress_apply(lambda text: 
                                          " ".join(token.lemma_ for token in nlp(text) 
                                                   if not token.is_stop and token.is_alpha))

100%|██████████| 20606/20606 [02:28<00:00, 138.73it/s]


In [13]:
from bertopic import BERTopic

In [23]:
from bertopic.representation import MaximalMarginalRelevance
from bertopic import BERTopic

# Create your representation model
representation_model = MaximalMarginalRelevance(diversity=0.3)

# Use the representation model in BERTopic on top of the default pipeline
topic_model2 = BERTopic(representation_model=representation_model)
topics, probs = topic_model2.fit_transform(clean_df_final_small['pr_transactiondescription'])

In [24]:
topic_model2.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,4312,-1_sealant_pulley_led_hose,"[sealant, pulley, led, hose, nissan, kit, valv...",[HANDHELD tape DISPENSER MAXIMUM TAPE WIDTH DI...
1,0,198,0_electric_channel_strongholdtm_advxicvcnym,"[electric, channel, strongholdtm, advxicvcnym,...","[MAYER ELECTRIC, MAYER ELECTRIC MAYER ELECTRIC..."
2,1,137,1_door_closer_access_bops,"[door, closer, access, bops, holes, hinge, ins...","[DOOR HARDWARE, door CLOSER CORBIN CLOSER SERI..."
3,2,112,2_lamps_lights_equipment_room,"[lamps, lights, equipment, room, hallways, lob...","[REXEL EQUIPMENT MOB LAMPS REXEL equipment, RE..."
4,3,109,3_chemtreat_reagent_permanganate_sample,"[chemtreat, reagent, permanganate, sample, che...","[PERMANGANATE REAGENT SAMPLE CHEMTREAT INC, PE..."
...,...,...,...,...,...
566,565,10,565_ledsgn_led_lamp_ledcorn,"[ledsgn, led, lamp, ledcorn, retrofit, lumen, ...","[LEDSGN LED M LAMP LIGHTING s, LEDSGN LED M G ..."
567,566,10,566_cfmw_ballast_iztpspsci_advft,"[cfmw, ballast, iztpspsci, advft, icfshldk, un...","[ADV ICNPNI ELE BALLAST FT s, ADV ICNPNI ELE B..."
568,567,10,567_ledpdkfl_dimmable_ledhopdkfl_ledebk,"[ledpdkfl, dimmable, ledhopdkfl, ledebk, blunt...",[TCP LEDPDKFL PAR FLOOD lead DIMMABLE K LIGHTI...
569,568,10,568_clamp_worm_gear_hose,"[clamp, worm, gear, hose, marine, stainless, b...","[WORM GEAR HOSE CLAMP INTERLOCKED MIN, WORM GE..."


In [25]:
topic_small2= topic_model2.reduce_topics(clean_df_final_small['pr_transactiondescription'], nr_topics=30)
topic_small2.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,4312,-1_electric_elevator_inspection_equipment,"[electric, elevator, inspection, equipment, fi...","[lead LAMP CITY ELECTRIC, ELECTRIC ELECTRIC, E..."
1,0,2660,0_bulb_lamp_ballast_fluorescent,"[bulb, lamp, ballast, fluorescent, wire, elect...",[PLUG cfl BULB T BULB SHAPE BULB BASE TYPE COM...
2,1,1887,1_valve_fitting_hose_kit,"[valve, fitting, hose, kit, psi, temp, gauge, ...",[MALE elbow TUBE FITTING MATERIAL low lead bra...
3,2,1841,2_chemtreat_boiler_contractors_industrial,"[chemtreat, boiler, contractors, industrial, c...",[COV HALL AIR TANK VA COMMONWEALTH DEPARTMENT ...
4,3,1416,3_filter_air_filters_tidewater,"[filter, air, filters, tidewater, merv, vacha,...",[pleat AIR FILTER STYLE AIR FILTERS panel perf...
5,4,1328,4_elevator_elev_inspection_testing,"[elevator, elev, inspection, testing, traction...",[inspection ELEV HYDRAULIC elevator HALL MIKE ...
6,5,1189,5_equipment_elec_wood_lowes,"[equipment, elec, wood, lowes, tractor, emhc, ...","[REXEL EQUIPMENT REXEL equipment, REXEL EQUIPM..."
7,6,984,6_fire_lp_protection_controls,"[fire, lp, protection, controls, inspection, h...",[PROVIDE FIRE EXTINGUISHER inspection BLDG ven...
8,7,806,7_head_screw_size_hex,"[head, screw, size, hex, steel, rpm, drill, fa...",[wood SCREW finish zinc plate MATERIAL STEEL D...
9,8,793,8_auto_tire_rotate_mileage,"[auto, tire, rotate, mileage, center, nissan, ...",[JMJK MARSHALL TIRE AUTO skilled PURCHASES MAR...


In [26]:
topic_small2.save("my_model_small", serialization="safetensors")

In [27]:
topic_info_df = topic_small2.get_topic_info()
topic_info_df

,Topic,Count,Name,Representation,Representative_Docs
0,-1,4312,-1_electric_elevator_inspection_equipment,"[electric, elevator, inspection, equipment, fi...","[lead LAMP CITY ELECTRIC, ELECTRIC ELECTRIC, E..."
1,0,2660,0_bulb_lamp_ballast_fluorescent,"[bulb, lamp, ballast, fluorescent, wire, elect...",[PLUG cfl BULB T BULB SHAPE BULB BASE TYPE COM...
2,1,1887,1_valve_fitting_hose_kit,"[valve, fitting, hose, kit, psi, temp, gauge, ...",[MALE elbow TUBE FITTING MATERIAL low lead bra...
3,2,1841,2_chemtreat_boiler_contractors_industrial,"[chemtreat, boiler, contractors, industrial, c...",[COV HALL AIR TANK VA COMMONWEALTH DEPARTMENT ...
4,3,1416,3_filter_air_filters_tidewater,"[filter, air, filters, tidewater, merv, vacha,...",[pleat AIR FILTER STYLE AIR FILTERS panel perf...
5,4,1328,4_elevator_elev_inspection_testing,"[elevator, elev, inspection, testing, traction...",[inspection ELEV HYDRAULIC elevator HALL MIKE ...
6,5,1189,5_equipment_elec_wood_lowes,"[equipment, elec, wood, lowes, tractor, emhc, ...","[REXEL EQUIPMENT REXEL equipment, REXEL EQUIPM..."
7,6,984,6_fire_lp_protection_controls,"[fire, lp, protection, controls, inspection, h...",[PROVIDE FIRE EXTINGUISHER inspection BLDG ven...
8,7,806,7_head_screw_size_hex,"[head, screw, size, hex, steel, rpm, drill, fa...",[wood SCREW finish zinc plate MATERIAL STEEL D...
9,8,793,8_auto_tire_rotate_mileage,"[auto, tire, rotate, mileage, center, nissan, ...",[JMJK MARSHALL TIRE AUTO skilled PURCHASES MAR...


In [14]:
topic_small= BERTopic.load("my_model_small")

In [15]:
topic_info_df = topic_small.get_topic_info()
topic_info_df

,Topic,Count,Name,Representation,Representative_Docs
0,-1,4312,-1_electric_elevator_inspection_equipment,"[electric, elevator, inspection, equipment, fi...",NaN
1,0,2660,0_bulb_lamp_ballast_fluorescent,"[bulb, lamp, ballast, fluorescent, wire, elect...",NaN
2,1,1887,1_valve_fitting_hose_kit,"[valve, fitting, hose, kit, psi, temp, gauge, ...",NaN
3,2,1841,2_chemtreat_boiler_contractors_industrial,"[chemtreat, boiler, contractors, industrial, c...",NaN
4,3,1416,3_filter_air_filters_tidewater,"[filter, air, filters, tidewater, merv, vacha,...",NaN
5,4,1328,4_elevator_elev_inspection_testing,"[elevator, elev, inspection, testing, traction...",NaN
6,5,1189,5_equipment_elec_wood_lowes,"[equipment, elec, wood, lowes, tractor, emhc, ...",NaN
7,6,984,6_fire_lp_protection_controls,"[fire, lp, protection, controls, inspection, h...",NaN
8,7,806,7_head_screw_size_hex,"[head, screw, size, hex, steel, rpm, drill, fa...",NaN
9,8,793,8_auto_tire_rotate_mileage,"[auto, tire, rotate, mileage, center, nissan, ...",NaN


In [33]:
df['transactiondescription']= df['transactiondescription'].astype(str).str.upper()

In [34]:
df['transactiondescription']= df['transactiondescription'].apply(remove_numbers)

In [35]:
def remove_words(text, words):
    if not isinstance(text, str):
        return text  # Return the original value if it's not a string
    # Create a regex pattern that matches any of the words to remove
    pattern = r'\b(?:' + '|'.join(re.escape(word) for word in words) + r')\b'
    # Use regex to remove the words
    cleaned_text = re.sub(pattern, '', text, flags=re.IGNORECASE)
    # Remove extra spaces
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    return cleaned_text

# Apply the function to the DataFrame column
df['clean_transactiondescription'] = df['transactiondescription'].apply(lambda x: remove_words(x, customize_stop_words))

In [36]:
def remove_words(text, words):
    if not isinstance(text, str):
        return text  # Return the original value if it's not a string
    # Create a regex pattern that matches any of the words to remove
    pattern = r'\b(?:' + '|'.join(re.escape(word) for word in words) + r')\b'
    # Use regex to remove the words
    cleaned_text = re.sub(pattern, '', text, flags=re.IGNORECASE)
    # Remove extra spaces
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    return cleaned_text

# Apply the function to the DataFrame column
df['clean_transactiondescription'] = df['transactiondescription'].apply(lambda x: remove_words(x, stop_words['words']))

In [39]:
df.head(50)

,sourcetransactionid,workorderid,wophaseid,transactiondate,transactiontype,transactionamount,transactiondescription,subledgertype,phdesc,phshop,phshopwithdesc,division,organization,orgdescription,shopzone,shopzonedescription,shopzonetype,clean_transactiondescription
2,614480,2120558,1,2014-01-06 12:23:36.660,INVENTORY MATERIAL RELEASE TO WORK ORDER,48.29,LITHONIA EMERGENCY UNIT-ELM DL,Material,HOUSING WEEKLY INSPECTIONS - EMMET HOUSE (EMME...,110,110 (FM-HOUSING MAINTENANCE 110),CH20131 FM-Operations,CC0537,CC0537 FM-Housing Maintenance,MZHOUSING2,FM-HOUSING MAINTENANCE AREA 2,MAINTENANCE,LITHONIA EMERGENCY UNIT-ELM DL
3,614481,2120559,1,2014-01-06 12:27:20.077,INVENTORY MATERIAL RELEASE TO WORK ORDER,48.29,LITHONIA EMERGENCY UNIT-ELM DL,Material,HOUSING WEEKLY INSPECTIONS - PAGE HOUSE (PAGE ...,110,110 (FM-HOUSING MAINTENANCE 110),CH20131 FM-Operations,CC0537,CC0537 FM-Housing Maintenance,MZHOUSING2,FM-HOUSING MAINTENANCE AREA 2,MAINTENANCE,LITHONIA EMERGENCY UNIT-ELM DL
4,614482,2120559,1,2014-01-06 12:27:20.200,INVENTORY MATERIAL RELEASE TO WORK ORDER,34.28,FIXTURE THREE-LIGHT -/ SANDSTONE MW #,Material,HOUSING WEEKLY INSPECTIONS - PAGE HOUSE (PAGE ...,110,110 (FM-HOUSING MAINTENANCE 110),CH20131 FM-Operations,CC0537,CC0537 FM-Housing Maintenance,MZHOUSING2,FM-HOUSING MAINTENANCE AREA 2,MAINTENANCE,FIXTURE THREE-LIGHT -/ SANDSTONE MW #
10,616159,2119390,1,2014-01-07 11:10:26.040,EXTERNAL CHARGE,38.63,"SUPPLIES, BUILDING R&M - FERGUSON ENTERPRISES ...",Material,HOUSING WEEKLY INSPECTIONS - LAMBETH FIELD #52...,141,141 (FM-ARTS PRECINCT ZONE (INACTIVE)),NaN,NaN,NaN,NaN,NaN,NaN,"SUPPLIES, BUILDING R&M - FERGUSON ENTERPRISES ..."
11,616160,2119390,1,2014-01-07 11:10:49.207,EXTERNAL CHARGE,52.84,"SUPPLIES, BUILDING R&M - FERGUSON ENTERPRISES ...",Material,HOUSING WEEKLY INSPECTIONS - LAMBETH FIELD #52...,141,141 (FM-ARTS PRECINCT ZONE (INACTIVE)),NaN,NaN,NaN,NaN,NaN,NaN,"SUPPLIES, BUILDING R&M - FERGUSON ENTERPRISES ..."
12,616543,2120550,1,2014-01-07 15:33:05.363,INVENTORY MATERIAL RELEASE TO WORK ORDER,48.29,LITHONIA EMERGENCY UNIT-ELM DL,Material,HOUSING WEEKLY INSPECTIONS - KENT HOUSE (KENT ...,110,110 (FM-HOUSING MAINTENANCE 110),CH20131 FM-Operations,CC0537,CC0537 FM-Housing Maintenance,MZHOUSING2,FM-HOUSING MAINTENANCE AREA 2,MAINTENANCE,LITHONIA EMERGENCY UNIT-ELM DL
13,616604,2120551,1,2014-01-07 16:10:36.923,INVENTORY MATERIAL RELEASE TO WORK ORDER,48.29,LITHONIA EMERGENCY UNIT-ELM DL,Material,HOUSING WEEKLY INSPECTIONS - DABNEY HOUSE (DAB...,110,110 (FM-HOUSING MAINTENANCE 110),CH20131 FM-Operations,CC0537,CC0537 FM-Housing Maintenance,MZHOUSING2,FM-HOUSING MAINTENANCE AREA 2,MAINTENANCE,LITHONIA EMERGENCY UNIT-ELM DL
24,620395,1506497,1,2014-01-09 14:30:01.197,INVENTORY MATERIAL RELEASE TO WORK ORDER,3.30,BLEACH CLOROX,Material,"PUMP,VACUUM NO.1 MCLEOD HALL BLDG.0105",371,371 (FM-HSPP ZONE 2 MECHANICAL 371),CH20132 FM-UVA Health Operations,CC0532,CC0532 FM-UVA Health Zone 2,MZHSPP2,HSPP MAINTENANCE ZONE 2,MAINTENANCE,BLEACH CLOROX
25,620396,1506497,1,2014-01-09 14:30:01.337,INVENTORY MATERIAL RELEASE TO WORK ORDER,14.85,WIPER LINT FREE,Material,"PUMP,VACUUM NO.1 MCLEOD HALL BLDG.0105",371,371 (FM-HSPP ZONE 2 MECHANICAL 371),CH20132 FM-UVA Health Operations,CC0532,CC0532 FM-UVA Health Zone 2,MZHSPP2,HSPP MAINTENANCE ZONE 2,MAINTENANCE,WIPER LINT FREE
26,620732,2120551,1,2014-01-09 15:49:40.880,INVENTORY MATERIAL RELEASE TO WORK ORDER,4.99,"RUBBER TEST END CAP """,Material,HOUSING WEEKLY INSPECTIONS - DABNEY HOUSE (DAB...,110,110 (FM-HOUSING MAINTENANCE 110),CH20131 FM-Operations,CC0537,CC0537 FM-Housing Maintenance,MZHOUSING2,FM-HOUSING MAINTENANCE AREA 2,MAINTENANCE,"RUBBER TEST END CAP """


In [41]:
df.reset_index(drop=True, inplace=True)

In [42]:
topic_final, prob_final = topic_small.transform(df['clean_transactiondescription'])

In [43]:
topic_final_series= pd.Series(topic_final)

prob_final_series= pd.Series(prob_final)

In [44]:
header_names = ['sourcetransactionid','transactiondescription', 'Topic', 'prob']

# Create a DataFrame
df_concat = pd.DataFrame({
    header_names[0]: df['sourcetransactionid'],
    header_names[1]: df['transactiondescription'],
    header_names[2]: topic_final_series,
    header_names[3]: prob_final_series
})

In [53]:
df_left_join = pd.merge(df_concat, topic_info_df, on='Topic', how='left')
df_left_join = df_left_join[['sourcetransactionid','Topic','prob','Name']]

In [46]:
print(df_left_join["Name"].value_counts().to_string())

Name
21_charge_fundtransfer_email_surcharge       59845
6_fire_lp_protection_controls                27570
2_chemtreat_boiler_contractors_industrial    24810
22_dodson_brothers_pest_renewal              19920
4_elevator_elev_inspection_testing            7491
3_filter_air_filters_tidewater                7110
12_cleaner_cleaning_cloth_brush               4465
0_bulb_lamp_ballast_fluorescent               4211
9_svcs_fleet_ptrusg_trane                     3839
11_belt_section_construction_thickness        3581
7_head_screw_size_hex                         3252
1_valve_fitting_hose_kit                      3024
-1_electric_elevator_inspection_equipment     2738
10_battery_batteries_standard_alkaline        2422
16_glove_gloves_disposable_nitrile            1382
5_equipment_elec_wood_lowes                   1344
8_auto_tire_rotate_mileage                    1256
13_tape_label_vinyl_marker                    1227
15_ceiling_floors_tile_builders                639
18_controls_carrier_contro

In [50]:
df_left_join.columns

Index(['sourcetransactionid', 'transactiondescription', 'Topic', 'prob',
       'Count', 'Name', 'Representation', 'Representative_Docs'],
      dtype='object')

In [56]:
df_final = pd.merge(df, df_left_join, on='sourcetransactionid', how='left')
df_final= df_final[['sourcetransactionid',
'workorderid',
'wophaseid',
'transactiondate',
'transactiontype',
'transactionamount',
'transactiondescription',
'subledgertype',
'phdesc',
'phshop',
'phshopwithdesc',
'division',
'organization',
'orgdescription',
'shopzone',
'shopzonedescription',
'shopzonetype',
 'Topic',
 'prob',
 'Name']]

In [1]:
import pandas as pd
df = pd.read_csv("topics_pm_txn_filt.csv")

/tmp/ipykernel_1007/496791296.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("topics_pm_txn_filt.csv")


In [12]:
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_colwidth', None) 

df[df["Name"]=="28_holiday_inn_th_hotel"].drop_duplicates().head(100)

,Unnamed: 0,sourcetransactionid,workorderid,wophaseid,transactiondate,transactiontype,transactionamount,transactiondescription,subledgertype,phdesc,...,phshopwithdesc,division,organization,orgdescription,shopzone,shopzonedescription,shopzonetype,Topic,prob,Name
1075,1075,790430,2135088,1,2014-03-27 08:40:30.610,EXTERNAL CHARGE,349.72,"SUPPLIES, BUILDING R&M - HOLIDAY INN NORTON - PO: - INVOICE:",Material,FIRE SUPPRESSION QUARTERLY - QUARTERLY TRAVEL EXPENSES FOR FIRE SYSTEMS PM AT UVACW (GENERAL),...,184 (FM-FIRE/LIFE SAFETY INSPECTIONS 184),CH20131 FM-Operations,CC0526,CC0526 FM-Fire Protection,NaN,NaN,NaN,28,0.522971,28_holiday_inn_th_hotel
1076,1076,790431,2135088,1,2014-03-27 08:40:33.557,EXTERNAL CHARGE,349.72,"SUPPLIES, BUILDING R&M - HOLIDAY INN NORTON - PO: - INVOICE:",Material,FIRE SUPPRESSION QUARTERLY - QUARTERLY TRAVEL EXPENSES FOR FIRE SYSTEMS PM AT UVACW (GENERAL),...,184 (FM-FIRE/LIFE SAFETY INSPECTIONS 184),CH20131 FM-Operations,CC0526,CC0526 FM-Fire Protection,NaN,NaN,NaN,28,0.522971,28_holiday_inn_th_hotel
1077,1077,790432,2135088,1,2014-03-27 08:40:35.717,EXTERNAL CHARGE,349.72,"SUPPLIES, BUILDING R&M - HOLIDAY INN NORTON - PO: - INVOICE:",Material,FIRE SUPPRESSION QUARTERLY - QUARTERLY TRAVEL EXPENSES FOR FIRE SYSTEMS PM AT UVACW (GENERAL),...,184 (FM-FIRE/LIFE SAFETY INSPECTIONS 184),CH20131 FM-Operations,CC0526,CC0526 FM-Fire Protection,NaN,NaN,NaN,28,0.522971,28_holiday_inn_th_hotel
1078,1078,790433,2135088,1,2014-03-27 08:40:38.040,EXTERNAL CHARGE,349.72,"SUPPLIES, BUILDING R&M - HOLIDAY INN NORTON - PO: - INVOICE:",Material,FIRE SUPPRESSION QUARTERLY - QUARTERLY TRAVEL EXPENSES FOR FIRE SYSTEMS PM AT UVACW (GENERAL),...,184 (FM-FIRE/LIFE SAFETY INSPECTIONS 184),CH20131 FM-Operations,CC0526,CC0526 FM-Fire Protection,NaN,NaN,NaN,28,0.522971,28_holiday_inn_th_hotel
53461,53461,4178702,2529432,1,2017-03-30 13:43:17.043,EXTERNAL CHARGE,3350.00,O'HILL DINING HALL- ALL INTERIOR & EXTERIOR WINDOW - BUILDING SERVICE INNOVAT - PO: - INVOICE:,Material,WINDOW CLEANING - WINDOWS (OHILL DINING FACILITY),...,770 (FM CUSTODIAL SERVICES MGMT 770),CH20131 FM-Operations,CC0523,CC0523 FM-Custodial,HKZ0,ZONE O CUSTODIAL ZONE,CUSTODIAL,28,0.452267,28_holiday_inn_th_hotel
56973,56973,4379806,2598795,1,2017-05-25 10:35:27.990,INVENTORY MATERIAL RELEASE TO WORK ORDER,14.50,WONDER BAR,Material,ZONE MAINTENANCE WEEKLY - BUILDING REPAIR CARTER HARRISON RESEARCH BLDG.1161 (CARTER-HARRISON RESEARCH BUILDING (MR-6)),...,371 (FM-HSPP ZONE 2 MECHANICAL 371),CH20132 FM-UVA Health Operations,CC0532,CC0532 FM-UVA Health Zone 2,MZHSPP2,HSPP MAINTENANCE ZONE 2,MAINTENANCE,28,0.278035,28_holiday_inn_th_hotel
61817,61817,4730269,2622573,1,2017-09-07 12:25:36.943,EXTERNAL CHARGE,350.00,HOTEL E AND RANGE DINING - INTERIOR AND EXTERIOR W - BUILDING SERVICE INNOVAT - PO: - INVOICE:,Material,WINDOW CLEANING - WINDOW CLEANING (HOTEL E ANNEX/ COLONNADE HOTEL),...,770 (FM CUSTODIAL SERVICES MGMT 770),CH20131 FM-Operations,CC0523,CC0523 FM-Custodial,HKZ0,ZONE O CUSTODIAL ZONE,CUSTODIAL,28,0.438909,28_holiday_inn_th_hotel
69166,69166,5260282,2703165,2,2018-02-12 09:59:41.727,INVENTORY MATERIAL RELEASE TO WORK ORDER,11.02,WONDER BAR,Material,"VACUUM PUMP MONTHLY - VACUUM,TANK MED.STACY HALL 1985 (CHECK) FRI.WKLY. Z-2 (STACEY HALL)",...,371 (FM-HSPP ZONE 2 MECHANICAL 371),CH20132 FM-UVA Health Operations,CC0532,CC0532 FM-UVA Health Zone 2,MZHSPP2,HSPP MAINTENANCE ZONE 2,MAINTENANCE,28,0.278035,28_holiday_inn_th_hotel
69167,69167,5260283,2703165,2,2018-02-12 09:59:41.807,INVENTORY MATERIAL RELEASE TO WORK ORDER,3.48,WONDER BAR,Material,"VACUUM PUMP MONTHLY - VACUUM,TANK MED.STACY HALL 1985 (CHECK) FRI.WKLY. Z-2 (STACEY HALL)",...,371 (FM-HSPP ZONE 2 MECHANICAL 371),CH20132 FM-UVA Health Operations,CC0532,CC0532 FM-UVA Health Zone 2,MZHSPP2,HSPP MAINTENANCE ZONE 2,MAINTENANCE,28,0.278035,28_holiday_inn_th_hotel
80778,80778,5952179,2778086,1,2018-08-17 09:21:03.510,EXTERNAL CHARGE,350.00,"-, HOTEL E/COLONNADE HOTEL WINDOW CLEANI - BUILDING SERVICE INNOVAT 

In [13]:
trxn_name_df = pd.read_csv("Trxn Names - Sheet1.csv")

In [19]:
trxn_name_df.columns

Index(['Name', 'Name_Clean'], dtype='object')

In [15]:
df_final = pd.read_csv("topics_pm_txn_filt.csv")

/tmp/ipykernel_1007/2191027113.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_final = pd.read_csv("topics_pm_txn_filt.csv")


In [21]:
df_final_export = pd.merge(df_final, trxn_name_df, on='Name', how='left')
df_final_export[['sourcetransactionid',
'workorderid',
'wophaseid',
'transactiondate',
'transactiontype',
'transactionamount',
'transactiondescription',
'subledgertype',
'phdesc',
'phshop',
'phshopwithdesc',
'division',
'organization',
'orgdescription',
'shopzone',
'shopzonedescription',
'shopzonetype',
 'Topic',
 'prob',
 'Name_Clean']].to_csv("topics_pm_txn_filt.csv")

In [22]:
df_final_export = pd.merge(df_final, trxn_name_df, on='Name', how='left')
df_final_export[['sourcetransactionid',
'workorderid',
'wophaseid',
'transactiondate',
'transactiontype',
'transactionamount',
'transactiondescription',
'subledgertype',
'phdesc',
'phshop',
'phshopwithdesc',
'division',
'organization',
'orgdescription',
'shopzone',
'shopzonedescription',
'shopzonetype',
 'Topic',
 'prob',
 'Name_Clean']].to_excel("topics_pm_txn_filt.xlsx")